In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.ndimage import gaussian_filter
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import random



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x_train = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/x_train.npy")/255.
x_test = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/x_test.npy")/255.
y_train_coarse = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/itba-rn-2245-cifar-100-2023q2/y_train_fine.npy")

In [ ]:
import pickle
with open("/kaggle/input/itba-rn-2245-cifar-100-2023q2/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/itba-rn-2245-cifar-100-2023q2/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [ ]:
from matplotlib import pyplot as plt
%load_ext tensorboard

Lo primero que hice fue aumentar la cantidad de datos. Esto lo hice por medio de darle la vuelta a las imágenes y tambien moví todos los pixels hacia la un lado en las imágenes normales y volteadas. Otra vez experimenté con añadir ruide de tipo 'salt and pepper', sin embargo, reducio el accuracy. 

In [ ]:
'''
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):

    noisy_image = np.copy(image)
    total_pixels = image.size

    # Add salt noise
    salt_pixels = int(total_pixels * salt_prob)
    salt_coordinates = [random.choice(range(total_pixels)) for _ in range(salt_pixels)]
    noisy_image.flat[salt_coordinates] = 255  # Set salt noise pixels to white

    # Add pepper noise
    pepper_pixels = int(total_pixels * pepper_prob)
    pepper_coordinates = [random.choice(range(total_pixels)) for _ in range(pepper_pixels)]
    noisy_image.flat[pepper_coordinates] = 0  # Set pepper noise pixels to black

    return noisy_image
x_salt_and_pepper = np.array([add_salt_and_pepper_noise(x_train[i], 0.01, 0.01) for i in range(len(x_train))])
'''

In [ ]:
#plt.imshow(x_salt_and_pepper[0])

In [ ]:
x_train_shift_left = np.array([np.roll(x_train[i],-2,axis=1) for i in range(len(x_train))])

In [ ]:
#Darle la vuelta a todos las imagenes --> despues se pegan con las otras imagenes
x_train_flip = np.array([np.fliplr(x_train[i]) for i in range(len(x_train))])
x_train_shiftflip_left = np.array([np.roll(x_train_flip[i],-2,axis=1) for i in range(len(x_train))])

In [ ]:
#Antes de utilizar el gray-scale se implementó tambien un Gaussian blur, tampoco causó un aumento sustancial
sigma = 1
blur = gaussian_filter(x_train[0],sigma=sigma)
plt.imshow(blur)
#x_train_blur = [gaussian_filter(x_train[i],sigma=sigma) for i in range(len(x_train))]

In [ ]:
y_train_coarse[0]

In [ ]:
labels_coarse[11]

In [ ]:
# Me fijo para una categoría de grano grueso, cuáles son las categorías de grano fino
coarse = 6
print(f"Grano grueso: {labels_coarse[coarse]}")
for fine in np.unique(y_train_fine[y_train_coarse == coarse]):
    print(f"Grano fino: {labels_fine[fine]}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

##CNN imports
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers.core import Dense, Flatten, Activation
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D, Input,Add
from tensorflow.keras.applications import MobileNet

Abajo se puede observar el transferlearning que intenté implementar. Al final no logré implementarlo por razones explicadas en la descripción de la red tipo resnet. 

In [ ]:
'''
# Load a pre-trained ResNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

#for layer in base_model.layers:
    #layer.trainable=False
'''

In [ ]:
#base_model.summary()

In [ ]:
#x_pre_train=base_model.predict(x)
#x_pre_val=base_model.predict(x_val)

In [ ]:
'''
input_preproc=Input(shape=(1,1,1024))

net=Flatten()(input_preproc)

net=Dense(units=256, activation='relu')(net)
net=Dropout(0.2)(net)
net=Dense(units=256, activation='relu')(net)
net=Dropout(0.2)(net)
net=Dense(units=100, activation = 'softmax')(net)

model=Model(input_preproc,net)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
'''

In [ ]:

'''

net_input = Input(shape=(32,32,3))

def inception_block(input_layer,filters=64):
    tower_1 = Conv2D(filters, (1,1), padding='same', activation='relu')(input_layer)
    tower_1 = Conv2D(filters, (3,3), padding='same', activation='relu')(tower_1)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_layer)
    tower_3 = Conv2D(filters, (1,1), padding='same', activation='relu')(tower_3)
    output = keras.layers.concatenate([tower_1, tower_3], axis = 3)
    return output


net=inception_block(net_input,filters=4)
net=BatchNormalization()(net)
net=inception_block(net,filters=4)
net=BatchNormalization()(net)
net=inception_block(net,filters=4)


net=Flatten()(net)

net=Dense(units=128, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=256, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=100, activation = 'softmax')(net)

model=Model(net_input,net)

model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()
'''

Esto es mi red convolucional ordinaria que está hecha completamente a mano. Fue añadiendo capas y filtros hasta llegar a un resultado deseado. Al final logré a obtener un accuracy de 48.3%.

In [ ]:
#Red ordinaria 
# Accuracy --> 48.3%

'''

dropout = 0.1
##Normaal CNN
model = Sequential()

model.add(Conv2D(filters=20, kernel_size=(5,5),input_shape=(32,32,3),padding='same', kernel_initializer=RandomNormal(mean=0.0,stddev=0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
##
model.add(Conv2D(filters=40, kernel_size=(5,5),padding='same', kernel_initializer=RandomNormal(mean=0.0,stddev=0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
##
model.add(Conv2D(filters=80, kernel_size=(5,5),padding='same', kernel_initializer=RandomNormal(mean=0.0,stddev=0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
##
model.add(Conv2D(filters=160, kernel_size=(3,3),padding='same', kernel_initializer=RandomNormal(mean=0.0,stddev=0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
##
model.add(Conv2D(filters=320, kernel_size=(3,3),padding='same', kernel_initializer=RandomNormal(mean=0.0,stddev=0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
##
model.add(Flatten())
model.add(Dense(units=200, activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units=200, activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units=200, activation='relu'))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units=100, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
'''


He intentado implementar transferlearning pero la red pre-entrenada que encontré no estaba bien ajustada a la red y ademas el output de esa read estaba encriptado de manera especifica lo cual necesitaria ajustar demasiado para ser util. 

En la resnet abajo cambié y quité unas capas y filtros para reducir la cantidad de parámetros.

In [ ]:
#Accuracy de 54% en tipo resnet

net_input = Input(shape=(32,32,3))

def res_block_first(input_layer,filters=16):
    
    #Primera capa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(aux_net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([aux_net,net])
    
    net=Activation("relu")(net)
    net=MaxPooling2D()(net)
    
    return net

def res_block(input_layer,filters=16):
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([input_layer,net])
    net=Activation("relu")(net)
    
    return net

def res_block_dec(input_layer,filters=32):
    
    input_layer=MaxPooling2D()(input_layer)
    net=Conv2D(filters=filters, kernel_size=(1, 1), padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Add()([aux_net,net])
    net=Activation("relu")(net)
    
    return net

net=res_block_dec(net_input,filters=32)
net=res_block(net,filters=32)
net=res_block(net,filters=32)

net=res_block_dec(net,filters=64)
net=res_block(net,filters=64)
net=res_block(net,filters=64)

net=res_block_first(net,filters=128)
net=res_block(net,filters=128)
net=res_block(net,filters=128)



net = AveragePooling2D() (net)

net=Flatten()(net)

net=Dense(units=1024,kernel_initializer=RandomNormal(mean=0.0,stddev=0.01), activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=100, activation = 'softmax')(net)

model=Model(net_input,net)

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
rlrop = ReduceLROnPlateau(
    monitor = "val_accuracy",
    factor = 0.5,
    patience = 3,
    verbose = 1,
    min_lr = 1e-5
)

In [ ]:
mc = ModelCheckpoint(
    "best_weights.hfd5",
    monitor = "val_accuracy",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
)

In [ ]:
es = EarlyStopping(
    monitor = "val_accuracy",
    patience = 4,
    verbose = 1,
    restore_best_weights = True,
)

In [ ]:
tb = TensorBoard(
    log_dir="logs",
)

In [ ]:
x_train2 =  np.concatenate((x_train, x_train_flip), axis=0)
x_train3 =  np.concatenate((x_train2, x_train_shift_left), axis=0) 
x_train4 =  np.concatenate((x_train3, x_train_shiftflip_left), axis=0) 

y_train_fine2 = np.concatenate((y_train_fine, y_train_fine), axis=0)
y_train_fine3 = np.concatenate((y_train_fine2, y_train_fine), axis=0)
y_train_fine4 = np.concatenate((y_train_fine3, y_train_fine), axis=0)
#Juntamos las imagenes normales y flipeadas y tambien duplicamos el train fine 


In [ ]:
x, x_val, y, y_val = train_test_split(x_train4,y_train_fine4, test_size=0.2, random_state=42, stratify=y_train_fine4)

In [ ]:
'''
total_model_output=model(base_model.output)
total_model=Model(base_model.input,total_model_output)
total_model.summary()
total_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
'''

In [ ]:
'''
for layer in base_model.layers:
    layer.trainable=True
total_model.summary()
'''

In [ ]:
history = model.fit(
    x = x, 
    y= y, 
    batch_size=64, 
    epochs=200, 
    validation_data = (x_val,y_val), 
    callbacks= [
        rlrop,
        es,
        mc,
        tb,
    ]
)

In [ ]:
last_valacc = history.history['val_accuracy'][-1]*100
print("Last Validation Accuracy:" , format(last_valacc, ".2f"), "%")
last_acc = history.history['accuracy'][-1]*100
print("Last Accuracy:" , format(last_acc, ".2f"), "%")

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])

In [ ]:
%tensorboard --logdir logs

In [ ]:
predictions = model.predict(x_test).argmax(axis=1)

In [ ]:
df = pd.DataFrame(predictions, columns=["Label"])

In [ ]:
df.index.name = "Id"

In [ ]:
df.head()

In [ ]:
df.to_csv("submission.csv")